In [1]:
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
import optuna

In [2]:
df = pd.read_csv('train.csv')
df_pred = pd.read_csv('test.csv', index_col='id')
sample = pd.read_csv('sample_submission.csv')

col = df.columns[1:]
df_more = pd.read_csv('Faults.NNA', header=None, sep='\t', names=col)

In [3]:
df

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,584,590,909972,909977,16,8,5,2274,113,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,1,808,816,728350,728372,433,20,54,44478,70,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,3,781,789,3353146,3353173,210,16,29,3202,114,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,4,1540,1560,618457,618502,521,72,67,48231,82,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19214,19214,749,757,143210,143219,17,4,4,2193,122,...,-0.1429,0.0044,0.2901,0,0,0,1,0,0,0
19215,19215,723,735,2488529,2488541,231,17,26,27135,104,...,0.7222,-0.0989,0.5378,0,0,0,0,0,0,1
19216,19216,6,31,1578055,1578129,780,114,98,71112,41,...,0.7719,-0.4283,0.9997,1,0,0,0,0,0,0
19217,19217,9,18,1713172,1713184,126,13,26,14808,88,...,0.9610,-0.1162,0.3509,0,0,0,0,0,0,1


In [4]:
df = pd.concat([df, df_more], axis=0)

In [5]:
df.drop(columns='id', inplace=True)

In [6]:
lst_target = ['Pastry', 'Z_Scratch', 'K_Scatch',
              'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']
X = df.drop(columns=lst_target)
y = np.argmax(df[lst_target], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# def objective(trial: optuna.Trial):
#     param = {
#             'random_seed': 42,
#             'learning_rate': 0.01805755995096796,
#             'iterations': 1894,
#             'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
#             'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
#             'random_strength': trial.suggest_float("random_strength", 1e-8, 10.0, log=True)
#             }

#     cat_model = CatBoostClassifier(**param, verbose=0)
#     cat_model.fit(X_train, y_train)
#     y_pred = cat_model.predict_proba(X_test)
#     return roc_auc_score(y_test, y_pred, multi_class="ovr")

In [8]:
# %%time
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

In [9]:
params = {
    'verbose': 0,
    'random_seed': 42,
    'learning_rate': 0.01805755995096796,
    'iterations': 1894,
    'l2_leaf_reg': 1.2363359603805641,
    'bagging_temperature': 0.5946780370917848,
    'random_strength': 1.2571795646000008
}

In [10]:
cat_model = CatBoostClassifier(**params)
cat_model.fit(X_train, y_train)
y_pred = cat_model.predict_proba(X_test)
print(roc_auc_score(y_test, y_pred, multi_class="ovr")) # 0.8819677271743828

0.8819677271743828


In [11]:
# cat_model.fit(X, y)
pred = cat_model.predict_proba(df_pred)

In [12]:
submission = pd.DataFrame(pred, columns=lst_target, index=df_pred.index)
submission.to_csv('cat_model.csv', index=True)

# kaggle = 0.89149